In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
import graphviz
import pydotplus
import io
from scipy import misc
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from __future__ import print_function
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer
from sklearn.metrics import confusion_matrix

In [2]:
%matplotlib inline

In [3]:
battles = pd.read_csv("battles.csv")
character_deaths = pd.read_csv("character-deaths.csv")
character_predictions = pd.read_csv("character-predictions.csv")

In [4]:
battles.info()
battles.isnull().any().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 25 columns):
name                  38 non-null object
year                  38 non-null int64
battle_number         38 non-null int64
attacker_king         36 non-null object
defender_king         35 non-null object
attacker_1            38 non-null object
attacker_2            10 non-null object
attacker_3            3 non-null object
attacker_4            2 non-null object
defender_1            37 non-null object
defender_2            2 non-null object
defender_3            0 non-null float64
defender_4            0 non-null float64
attacker_outcome      37 non-null object
battle_type           37 non-null object
major_death           37 non-null float64
major_capture         37 non-null float64
attacker_size         24 non-null float64
defender_size         19 non-null float64
attacker_commander    37 non-null object
defender_commander    28 non-null object
summer                37 non-null flo

20

In [5]:
character_deaths.describe()

,Death Year,Book of Death,Death Chapter,Book Intro Chapter,Gender,Nobility,GoT,CoK,SoS,FfC,DwD
count,305.000000,307.000000,299.000000,905.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000
mean,299.157377,2.928339,40.070234,28.861878,0.828790,0.468920,0.272628,0.353326,0.424209,0.272628,0.284624
std,0.703483,1.326482,20.470270,20.165788,0.376898,0.499305,0.445554,0.478264,0.494492,0.445554,0.451481
min,297.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,299.000000,2.000000,25.500000,11.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,299.000000,3.000000,39.000000,27.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,300.000000,4.000000,57.000000,43.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,300.000000,5.000000,80.000000,80.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
character_deaths.head()

,Name,Allegiances,Death Year,Book of Death,Death Chapter,Book Intro Chapter,Gender,Nobility,GoT,CoK,SoS,FfC,DwD
0,Addam Marbrand,Lannister,NaN,NaN,NaN,56.0,1,1,1,1,1,1,0
1,Aegon Frey (Jinglebell),None,299.0,3.0,51.0,49.0,1,1,0,0,1,0,0
2,Aegon Targaryen,House Targaryen,NaN,NaN,NaN,5.0,1,1,0,0,0,0,1
3,Adrack Humble,House Greyjoy,300.0,5.0,20.0,20.0,1,1,0,0,0,0,1
4,Aemon Costayne,Lannister,NaN,NaN,NaN,NaN,1,1,0,0,1,0,0


In [7]:
train, test = train_test_split(character_deaths, test_size = 0.10,random_state=41)
print("Training size: {}; Test size {}".format(len(train), len(test)))

Training size: 825; Test size 92


In [8]:
c = DecisionTreeClassifier(min_samples_split=20,random_state=41)

In [9]:
features = ["Death Chapter","Book Intro Chapter","CoK","SoS"]
X_train = train[features]
featureso = ["Gender","Nobility","FfC","DwD"]
y_train = train[featureso]
X_test = test[features]
y_test = test[featureso]

In [18]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(X_train)
impo = Imputer(missing_values='NaN', strategy='mean', axis=0)
impo = imp.fit(X_test)
X_train_imp = imp.transform(X_train)
X_test_imp = impo.transform(X_test)
dt = c.fit(X_train_imp, y_train)

In [19]:
y_pred = c.predict(X_test_imp)
score = accuracy_score(y_test,y_pred)*100
print("Accuracy", round(score,1),"% ")

Accuracy 32.6 % 


In [20]:
score_pairs={}
for n in range(10,110,1):
    temp = RandomForestClassifier(n_estimators = 100)
    RandomFor = temp.fit(X_train_imp, y_train)
    forest_y_pred = temp.predict(X_test_imp)
    score = accuracy_score(y_test, forest_y_pred) * 100
    rounded_score = round(score, 1)
    score_pairs[n]=rounded_score

In [21]:
print(RandomFor)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [22]:
print ("Train Accuracy :: ", accuracy_score(y_train, RandomFor.predict(X_train_imp)))
print ("Test Accuracy  :: ", accuracy_score(y_test, RandomFor.predict(X_test_imp)))
print( " Confusion matrix\n", confusion_matrix(y_test.values.argmax(axis=1), RandomFor.predict(X_test_imp).argmax(axis=1)))

Train Accuracy ::  0.696969696969697
Test Accuracy  ::  0.40217391304347827
 Confusion matrix
 [[75  3  1  1]
 [ 8  1  0  0]
 [ 1  0  0  0]
 [ 1  0  0  1]]
